In [12]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
import json

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
n_List = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    n_List.append(row.text)

In [4]:
df = pd.DataFrame({"Neighborhood": n_List})
df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [5]:
df.shape

(71, 1)

In [13]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="my-app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [19]:
l = []
for neighborhood in df["Neighborhood"].tolist():
    location = geocode(neighborhood)
    l.append([neighborhood,location.latitude,location.longitude])
df_cor = pd.DataFrame(l,columns=['Neighborhood','Latitude','Longitude'])
df = pd.merge(df,df_cor,on='Neighborhood',how='left')
df.head()

,Neighborhood,Latitude,Longitude
0,Alam Damai,3.064708,101.740735
1,"Ampang, Kuala Lumpur",3.152135,101.732250
2,Bandar Menjalara,3.194136,101.633634
3,Bandar Sri Permaisuri,3.101158,101.710872
4,Bandar Tasik Selatan,3.076097,101.711447


In [20]:
address = 'Kuala Lumpur, Malaysia'
location = geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Kuala Lumpur, Malaysia are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Kuala Lumpur, Malaysia are 3.1516964, 101.6942371.


In [21]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11,tiles="Stamen Toner")

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [24]:
# define Foursquare Credentials and Version and explore the neighborhoods
CLIENT_ID = 'PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H' # your Foursquare ID
CLIENT_SECRET = '0KYAH2JYI2AEWBAKVWY4Y1BYSS2UMVJLTHQA0QK5MC2SRICD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H
CLIENT_SECRET:0KYAH2JYI2AEWBAKVWY4Y1BYSS2UMVJLTHQA0QK5MC2SRICD


In [26]:
#Top 100 venues that are within a radius of 2000 meter
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
venues_df = pd.DataFrame(venues,columns=['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory'])
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.064708,101.740735,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.064708,101.740735,Alam Damai Recreational Park,3.066424,101.740265,Other Great Outdoors
2,Alam Damai,3.064708,101.740735,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
3,Alam Damai,3.064708,101.740735,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House
4,Alam Damai,3.064708,101.740735,"Alam Damai, Damai Jasa",3.069329,101.744512,Park


In [27]:
venues_df.groupby(["Neighborhood"]).count() #Number of venues returned for each neighbor remember max limit was 100

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
...,...,...,...,...,...,...
Taman Tun Dr Ismail,100,100,100,100,100,100
Taman U-Thant,100,100,100,100,100,100
Taman Wahyu,100,100,100,100,100,100


We have fetched the data of 100 top venues for each neighbours(n = 71;df.shape[0]). Now look how many unique categories are there in total of all top venues for each neighbours. 

In [28]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))  

There are 320 unique categories.


In [29]:
venues_df['VenueCategory'].shape #It should be 7100(71 x 100) if all neighbours have 100 top venues data

(6786,)

In [30]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6786, 321)


,Neighborhoods,Accessories Store,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,...,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
# group rows by neighborhood taking the mean of the frequency of occurrence of each category
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(kl_grouped.shape)
kl_grouped

(70, 321)


,Neighborhoods,Accessories Store,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,...,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Alam Damai,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.01,0.00,0.03,0.01,0.00,0.0,0.00,0.00,0.00,0.0,0.01,0.00,0.01,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.02,...,0.00,0.00,0.01,0.00,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
1,"Ampang, Kuala Lumpur",0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.02,0.00,0.00,0.00,0.04,0.01,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.02,0.01,0.00,0.00,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.00,0.01,0.00,0.0,0.04,...,0.00,0.01,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
2,Bandar Menjalara,0.00,0.01,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.0,0.00,0.01,0.00,0.0,0.00,0.00,0.01,0.03,0.00,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.00,0.00,0.00,0.0,0.00,...,0.00,0.01,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.03,0.00,0.00,0.01,0.00,0.0,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.03,0.0,0.00,0.00,0.00,0.0,0.0,0.01,0.00,0.0,0.00,0.0
3,Bandar Sri Permaisuri,0.01,0.00,0.0,0.0,0.00,0.0,0.00,0.01,0.00,0.00,0.01,0.08,0.00,0.01,0.0,0.01,0.00,0.01,0.0,0.00,0.00,0.03,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.01,...,0.00,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.02,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.01,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
4,Bandar Tasik Selatan,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.01,0.00,0.01,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.01,0.00,0.0,0.02,...,0.00,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.05,0.00,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Taman Tun Dr Ismail,0.01,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.02,0.01,0.01,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.01,0.01,0.00,0.0,0.00,...,0.00,0.00,0.00,0.00,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.03,0.00,0.01,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.01,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.01,0.0
66,Taman U-Thant,0.00,0.00,0.0,0.0,0.01,0.0,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.02,0.00,0.00,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.01,0.0,0.00,0.02,

**Create a new DataFrame for Shopping Mall data only**

In [32]:
df_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
print(df_mall.shape)
df_mall.head()

(70, 2)


,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.01
2,Bandar Menjalara,0.00
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.01


Cluster Neighborhoods


In [34]:
kclusters = 3

kl_clustering = df_mall.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 0, 2, 1, 1, 0, 2], dtype=int32)

In [35]:
kl_merged = df_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [36]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,2
1,"Ampang, Kuala Lumpur",0.01,0
2,Bandar Menjalara,0.00,2
3,Bandar Sri Permaisuri,0.00,2
4,Bandar Tasik Selatan,0.01,0


In [37]:
kl_merged = kl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() 

(70, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,2,3.064708,101.740735
1,"Ampang, Kuala Lumpur",0.01,0,3.152135,101.732250
2,Bandar Menjalara,0.00,2,3.194136,101.633634
3,Bandar Sri Permaisuri,0.00,2,3.101158,101.710872
4,Bandar Tasik Selatan,0.01,0,3.076097,101.711447


In [39]:
# sort the results by Cluster Labels
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
69,Wangsa Maju,0.010000,0,3.205667,101.731908
23,Dang Wangi,0.020000,0,3.156686,101.701975
51,Sri Hartamas,0.010000,0,3.156072,101.646406
63,Taman Sri Sinar,0.010000,0,3.188012,101.649720
21,Damansara Town Centre,0.020408,0,3.021152,101.586693
...,...,...,...,...,...
58,Taman Len Seng,0.000000,2,3.074114,101.742450
57,Taman Ibukota,0.000000,2,3.211147,101.715124
24,Desa Petaling,0.000000,2,3.084185,101.703552
3,Bandar Sri Permaisuri,0.000000,2,3.101158,101.710872


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11,tiles='stamen toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters.save('cluster.html')
map_clusters

Examine the Clusters

#### Cluster 0

In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
69,Wangsa Maju,0.010000,0,3.205667,101.731908
23,Dang Wangi,0.020000,0,3.156686,101.701975
51,Sri Hartamas,0.010000,0,3.156072,101.646406
63,Taman Sri Sinar,0.010000,0,3.188012,101.649720
21,Damansara Town Centre,0.020408,0,3.021152,101.586693
12,Bukit Bintang,0.020000,0,3.147107,101.708601
66,Taman U-Thant,0.020000,0,3.156499,101.729820
37,Maluri,0.020000,0,3.123228,101.726900
16,Bukit Petaling,0.010000,0,3.130081,101.696380
18,"Cheras, Kuala Lumpur",0.011905,0,3.112650,101.714296


#### Cluster 1

In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
15,Bukit Nanas,0.03,1,3.156180,101.704833
11,Brickfields,0.03,1,3.128857,101.684553
25,"Federal Hill, Kuala Lumpur",0.04,1,3.139850,101.678535
22,"Damansara, Kuala Lumpur",0.03,1,3.143285,101.662347
7,Bangsar Park,0.05,1,3.130830,101.669440
6,Bangsar,0.05,1,3.130830,101.669440
42,"Pudu, Kuala Lumpur",0.03,1,3.134931,101.712021
65,Taman Tun Dr Ismail,0.03,1,3.150808,101.624919
40,Mont Kiara,0.03,1,3.169999,101.652147


#### Cluster 2

In [43]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
46,Semarak,0.0,2,-8.258950,115.162471
44,Salak South,0.0,2,3.091532,101.703433
41,Pantai Dalam,0.0,2,3.095631,101.670049
35,Kuchai Lama,0.0,2,3.089438,101.684676
32,Kampung Padang Balang,0.0,2,3.216692,101.694328
26,Happy Garden,0.0,2,4.736131,115.000413
14,Bukit Kiara,0.0,2,3.158462,101.636003
52,Sri Petaling,0.0,2,3.061494,101.687076
13,Bukit Jalil,0.0,2,3.058453,101.687439
10,"Batu, Kuala Lumpur",0.0,2,3.201823,101.671022


###Quick Result


*   Mostly shopping malls are situated in cluster 1 ranging from 3 to 5 and hence highest competition.
*   Cluster 2 has least competition with no present situated.
*   Cluster 0 has medium level of competition having malls ranging from 1 to 2.

